In [74]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import count_distinct
from pyspark.sql.functions import count
from pyspark.sql.functions import round
from pyspark.sql.functions import sum

In [75]:
df = pd.read_excel("online_retail.xlsx")
df.to_csv('online_retail.csv', index=False)

In [76]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [84]:
# Создание датафрейма
schema = StructType([
    StructField("InvoiceNo", StringType(), True),
    StructField("StockCode", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("InvoiceDate", TimestampNTZType(), True),
    StructField("UnitPrice", DoubleType(), True),
    StructField("CustomerID", FloatType(), True),
    StructField("Country", StringType(), True),
    ])

df = spark.read.csv('online_retail.csv', sep = ",", header=True, schema=schema)

In [85]:
#Количество строк
df.count()

541909

In [87]:
#Уникальные клиенты
df.select(count_distinct("CustomerID")).show()

+--------------------------+
|count(DISTINCT CustomerID)|
+--------------------------+
|                      4372|
+--------------------------+



In [88]:
#Страна с максимальными продажами
df.select("Country").groupby("Country").count().sort("count", ascending=False).limit(1).show()

+--------------+------+
|       Country| count|
+--------------+------+
|United Kingdom|495478|
+--------------+------+



In [89]:
#Самая ранняя покупка
df.agg({'InvoiceDate':'min'}).show()

+-------------------+
|   min(InvoiceDate)|
+-------------------+
|2010-12-01 08:26:00|
+-------------------+



In [90]:
#Самая поздняя покупка
df.agg({'InvoiceDate':'max'}).show()

+-------------------+
|   max(InvoiceDate)|
+-------------------+
|2011-12-09 12:50:00|
+-------------------+



In [91]:
def agg_count_df(df):
    df.agg(*[count(c).alias(c) for c in df.columns]).show()

In [92]:
agg_count_df(df)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   541909|   541909|     540455|  541909|     541909|   541909|    406829| 541909|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [93]:
# Удалим записи с нулевыми значениями из набора данных чтобы количество записей было ровное
df_clean = df.dropna(how='any')
agg_count_df(df_clean)

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|   406829|   406829|     406829|  406829|     406829|   406829|    406829| 406829|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [94]:
recency = df_clean.withColumn('recency', F.datediff(F.current_date(), df_clean.InvoiceDate))

In [95]:
# расчет показателя "frequency", создание нового датасета.
frequency = df_clean.groupBy('CustomerID', 'InvoiceNo').count().\
                           groupBy('CustomerID').agg(count("*").alias("frequency"))
agg_count_df(frequency)
frequency.show(5)

+----------+---------+
|CustomerID|frequency|
+----------+---------+
|      4372|     4372|
+----------+---------+

+----------+---------+
|CustomerID|frequency|
+----------+---------+
|   17323.0|        9|
|   13999.0|       15|
|   15512.0|        2|
|   12714.0|        5|
|   16156.0|       13|
+----------+---------+
only showing top 5 rows



In [96]:
# расчет значения "TotalPrice", создание нового столбца с данными значениями
total_price_clean = df_clean.withColumn('TotalPrice', round( df.Quantity * df.UnitPrice, 2 ) )

In [97]:
# расчет показателя "monetary", создание нового датасета
monetary = total_price_clean.groupBy("CustomerID").agg(round(sum('TotalPrice'),2).alias('monetary'))
monetary.show(5)

+----------+--------+
|CustomerID|monetary|
+----------+--------+
|   13999.0| 3865.26|
|   15512.0|   121.0|
|   13305.0| 2000.86|
|   15640.0|12433.34|
|   13094.0| 1708.86|
+----------+--------+
only showing top 5 rows



In [98]:
# добавление новых столбцов в датасет
total_price_new = recency.join(frequency, 'CustomerID', how = 'inner').join(monetary, 'CustomerID', how = 'inner')
total_price_new.show(5)

+----------+---------+---------+--------------------+--------+-------------------+---------+--------------+-------+---------+--------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|       Country|recency|frequency|monetary|
+----------+---------+---------+--------------------+--------+-------------------+---------+--------------+-------+---------+--------+
|   17850.0|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|United Kingdom|   4659|       35| 5288.63|
|   17850.0|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|United Kingdom|   4659|       35| 5288.63|
|   17850.0|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|United Kingdom|   4659|       35| 5288.63|
|   17850.0|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|United Kingdom|   4659|       35| 5288.63|
|   17850.0|   536365|   84029E|RED WOOLLY HOTTIE...|  

In [99]:
total_price_new.select('recency','frequency','monetary').summary().show()

+-------+-----------------+-----------------+------------------+
|summary|          recency|        frequency|          monetary|
+-------+-----------------+-----------------+------------------+
|  count|           406829|           406829|            406829|
|   mean|4437.863667535992|23.22480698278638|11067.338028559558|
| stddev|112.8505892969814| 45.4994555163778|30144.694797603934|
|    min|             4286|                1|          -4287.63|
|    25%|             4336|                4|           1084.34|
|    50%|             4417|                8|           2616.32|
|    75%|             4533|               18|            6147.4|
|    max|             4659|              248|         279489.02|
+-------+-----------------+-----------------+------------------+



In [100]:
total_price_new = total_price_new.withColumn('recency_group', F.when(F.col("recency")>4409, "C")\
                                            .when(F.col('recency') > 4325, "B")\
                                            .otherwise("A"))
total_price_new = total_price_new.withColumn('frequency_group', F.when(F.col("frequency")< 8, "C")\
                                            .when(F.col('frequency') < 18, "B")\
                                            .otherwise("A"))
total_price_new = total_price_new.withColumn('monetary_group', F.when(F.col("monetary")< 2616, "C")\
                                            .when(F.col('monetary') < 6147, "B")\
                                            .otherwise("A"))
total_price_new.show(5)

+----------+---------+---------+--------------------+--------+-------------------+---------+--------------+-------+---------+--------+-------------+---------------+--------------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|       Country|recency|frequency|monetary|recency_group|frequency_group|monetary_group|
+----------+---------+---------+--------------------+--------+-------------------+---------+--------------+-------+---------+--------+-------------+---------------+--------------+
|   17850.0|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|United Kingdom|   4659|       35| 5288.63|            C|              A|             B|
|   17850.0|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|United Kingdom|   4659|       35| 5288.63|            C|              A|             B|
|   17850.0|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|United K

In [101]:
total_price_new = total_price_new.withColumn('groups',
                                            F.concat(F.col('recency_group'),F.col('frequency_group'),F.col('monetary_group')))
total_price_new.select('CustomerID','recency','frequency','monetary','recency_group','frequency_group','monetary_group', 'groups').show(5)

+----------+-------+---------+--------+-------------+---------------+--------------+------+
|CustomerID|recency|frequency|monetary|recency_group|frequency_group|monetary_group|groups|
+----------+-------+---------+--------+-------------+---------------+--------------+------+
|   17323.0|   4301|        9|  908.99|            A|              B|             C|   ABC|
|   17323.0|   4301|        9|  908.99|            A|              B|             C|   ABC|
|   17323.0|   4301|        9|  908.99|            A|              B|             C|   ABC|
|   17323.0|   4305|        9|  908.99|            A|              B|             C|   ABC|
|   17323.0|   4305|        9|  908.99|            A|              B|             C|   ABC|
+----------+-------+---------+--------+-------------+---------------+--------------+------+
only showing top 5 rows



In [102]:
result = total_price_new.select(['CustomerID']).filter(total_price_new.groups == 'AAA').distinct()
result.show(5)

+----------+
|CustomerID|
+----------+
|   17315.0|
|   16746.0|
|   12748.0|
|   15039.0|
|   16729.0|
+----------+
only showing top 5 rows



In [103]:
result.toPandas().to_csv('result.csv', index = False)